<h1> Introduction </h1>

This notebook demonstrates how to scale an implemenation of Full Waveform Inversion using multiple FPGAs with bitstreams containing one or multiple copied compute units.

<h3> Define experiment parameters: </h3>
    
XCLBIN_PATH_DEFAULT => Default path for the .xclbin file containing 1 compute unit  
XCLBIN_PATH_MULTCU => Default path for .xclbin file containing 2 copied compute units  
XRT_ENV_PATH => Path to the Xilinx Runtime setup script.  
DEVICE_NAME_DEFAULT => Default name for the FPGA device  
SPAWN_PATH => Location on the host machines where the worker is spawned.
DIR_PATH => Path to the directory containing the FWI input files. 

In [4]:
XCLBIN_PATH_DEFAULT = "bitstreams/u280_xclbin/500_500_HBM/FullW.xclbin"
XCLBIN_PATH_MULTCU = "bitstreams/u280_xclbin/500_250_HBM/FullW.xclbin"
XRT_ENV_PATH = "/opt/xilinx/xrt/setup.sh"
DEVICE_NAME_DEFAULT="xilinx_u280_xdma_201920_3"
SPAWN_PATH = "/mnt/scratch/ldierick/octoray/fwi"
DIR_PATH_FWI = "default/"

If there are more than 1 host, the connect_options and worker_options can be lists of dictionaries containing the different specifications per worker. The list and hosts are matched in the order they are specified and the lists must be the same length.  

The pynqimport.py file reads the bitstream from config["overlay"] and downloads it to the host machine.

**Write the config to the cluster_config.json file before instantiating the workers, or the Overlay might not match** 

In [5]:
import json
config = {
    "scheduler":"10.1.212.129",
    "hosts":["10.1.212.126","10.1.212.127"],
    "connect_options":{"port":22,"xrt":XRT_ENV_PATH,"dir":SPAWN_PATH},
    "worker_options":{"nthreads":1,"n_workers":1,"preload":"pynqimport.py","nanny":"0","memory_limit":0},
    "scheduler_options":{"port":8786},
    "worker_class":"distributed.Worker",
    "overlay": XCLBIN_PATH_DEFAULT
}

with open("cluster_config.json","w") as f:
    json.dump(config,f)

<h3> Define the worker method </h3> 

Here, we define the Python method which will be executed on each of the Dask workers. This function calls the driver using the data partition it receives, and returns the output data (along with some performance statistics) to the caller (the Dask client). 


In [2]:
def execute_function(grid_data,kernel,id):
    import numpy as np
    import time
    from pynq import Overlay, allocate, Device, lib
    from FWIDriver import FWI
    
    start_time = time.time()
    
    # Set up the configuration
    cu = kernel["compute_unit"]
    config = kernel["config"]
    path = kernel["path_to_kernel"]
    grid_data = grid_data
    resolution = config["Freq"]["nTotal"] * config["nSources"] * config["nReceivers"]
    gridsize = len(grid_data)   
    config["tolerance"] = 9.99*10**-7
    config["max"] = 1000

    # Load the overlay
    devices = Device.devices
    
    # Get the Overlay
    #TODO: add device name to config file
    ol = Overlay(path, download=False, device=devices[0])

    # Allocate the buffers
    A = allocate(shape=(resolution,gridsize), dtype=np.complex64, target=getattr(ol,kernel["functions"][0]["dotprod_"+str(cu)][0]))
    B = allocate(shape=(gridsize,), dtype=np.float32, target=getattr(ol,kernel["functions"][0]["dotprod_"+str(cu)][1]))
    C = allocate(shape=(resolution,), dtype=np.complex64, target=getattr(ol,kernel["functions"][0]["dotprod_"+str(cu)][2]))

    D = allocate(shape=(resolution,gridsize), dtype=np.complex64,  target=getattr(ol,kernel["functions"][1]["update_"+str(cu)][0]))
    E = allocate(shape=(resolution,),dtype=np.complex64,  target=getattr(ol,kernel["functions"][1]["update_"+str(cu)][1]))
    F = allocate(shape=(gridsize), dtype=np.complex64, target=getattr(ol,kernel["functions"][1]["update_"+str(cu)][2]))

    # set up the kernel IP's
    dotprod = getattr(ol,"dotprod_"+str(cu))
    update = getattr(ol,"update_"+str(cu))
    
    
    # Execute the Full Waveform Inversion algorithm
    fwi = FWI(A,B,C,D,E,F,dotprod,update,config,resolution,gridsize,True)

    fwi.pre_process(grid_data)

    # reconstruct the grid by performing Full Wavefrom Inversion
    chi = fwi.reconstruct()

    total_time = time.time() - start_time

    # free all the buffers
    A.freebuffer()
    B.freebuffer()
    C.freebuffer()
    D.freebuffer()
    E.freebuffer()
    F.freebuffer()
        
    # Return statistics and results from FWI
    
    dict_t = {
    "id": id,
    "cu": cu,
    "time": total_time,
    "chi":chi,
    }
    return dict_t
    


<h2> SSH helper function </h2>

https://www.ssh.com/academy/ssh/copy-id

Sometimes we need to connect with a password once. We can use this helper function to login to all the hosts once and use passwordless authentication afterwards.

In [ ]:
import asyncio, asyncssh
import json
    
for h in config["hosts"]:
    async with asyncssh.connect(h,22,password="",username="") as conn:
        res = await conn.run("uname")
        print(res.stdout,end='')
                

<h2> Set up Octoray </h2>

Set up the Octoray framework. A user can pass either a dict or a config file containing a dict and specifiy if the scheduler and workers need to be set up or are already instantiated manually.

In [ ]:
from Octoray import Octoray

# Create an octoray instance with the 
octoray = Octoray(ssh_cluster=True,config_file=config)


# first load in the data
import json
import copy
import time

# Load in data and config settings
data = []
with open(DIR_PATH_FWI+"input/"+"10x10_100"+".txt") as f:
    for l in f:
        data.append(float(l))
        
data.extend(data)

fwi_config = None
with open(DIR_PATH_FWI+"input/GenericInput.json") as f:
    fwi_config = json.load(f)

#set specific configurations for different types of kernels
single_cu_config = fwi_config
double_cu_config = copy.deepcopy(fwi_config)
single_cu_config["ngrid"]["x"]=50
double_cu_config["ngrid"]["x"]=25

# Configure the kernels by specifying the path to the bitstream, number of compute units, batchsize per compute unit and the function names and variables with their respective memory banks.
single_cu = octoray.create_kernel(XCLBIN_PATH_DEFAULT,1,500,[[{"dotprod_1":["HBM0","HBM1","HBM2"]},{"update_1":["HBM3","HBM4","HBM5"]}]],single_cu_config)

double_cu = octoray.create_kernel(XCLBIN_PATH_MULTCU,2,250,[[{"dotprod_1":["HBM0","HBM1","HBM2"]},{"update_1":["HBM6","HBM7","HBM8"]}],
                                                [{"dotprod_2":["HBM3","HBM4","HBM5"]},{"update_2":["HBM9","HBM10","HBM11"]}]],double_cu_config)

# Finally, add the kernels you want to execute, in this case we add one single compute unit kernel per host.
kernels = []
for i in range(len(octoray.hosts)):
    kernels.append(copy.deepcopy(double_cu))
    
data_split, kernels_split = octoray.setup_cluster(data,*kernels)


In [ ]:
from dask.distributed import Client, progress

client = Client("tcp://10.1.212.127:8786")
client

<h1> Execute the added kernels on the host machines </h1>

In [ ]:
import time
t = time.time()

result = octoray.execute_hybrid(execute_function,data_split,kernels_split)

print(result)
print(time.time()-t)


<h3> Graceful shutdown for OpenSSH version >= 7.9 </h3>
 
Doesn't work on XACC yet... (UNTESTED)

In [ ]:
octoray.shutdown()

<h3> Ungraceful shutdown for OpenSSH version < 7.9 </h3>

In [ ]:
await octoray.fshutdown()